In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import os
from django.conf import settings
FAISS_DB_PATH = "./school_images_index"

# FAISS 데이터베이스 불러오기
embeddings = OpenAIEmbeddings(openai_api_key=key)
vectorstore = FAISS.load_local(FAISS_DB_PATH, embeddings,allow_dangerous_deserialization=True)


NameError: name 'key' is not defined

In [ ]:
# 유사도 검색 수행
search_query = "재밌는 그림"
results = vectorstore.similarity_search_with_score(search_query, k=10)  # 상위 10개 결과 검색

# 결과 출력을 위한 리스트 생성
similar_images = []
for doc, score in results:
    # FAISS의 score는 거리(작을수록 유사)이므로 유사도로 변환
    similarity = 1 / (1 + score)  # 점수를 0~1 사이 값으로 정규화
    
    # 결과를 딕셔너리로 저장
    image_info = {
        'title': doc.metadata.get('title', 'No Title'),
        'author': doc.metadata.get('author', 'Unknown'),
        'similarity_score': round(similarity, 3),
        'content': doc.page_content[:100] + '...' if len(doc.page_content) > 100 else doc.page_content
    }
    similar_images.append(image_info)

# 유사도 점수로 정렬
similar_images = sorted(similar_images, key=lambda x: x['similarity_score'], reverse=True)

# 결과 출력
print("\n=== 재밌는 그림 유사도 검색 결과 ===")
for i, img in enumerate(similar_images, 1):
    print(f"\n{i}. {img['title']}")
    print(f"작가: {img['author']}")
    print(f"유사도 점수: {img['similarity_score']}")
    print(f"설명: {img['content']}")
    print("-" * 50)